---
---
---
---
---

### **Her forward içinde, belirli katmanlar arasında kontrollü bilgi birleşimi yapılır.**

Forward içinde:

✔️ C5 bilgisi C4’e gider

✔️ C4 bilgisi C3’e gider

✔️ C3 bilgisi C2’ye gider

---
---
---
---



# Feature Pyramid Network (FPN) — Detaylı Anlatım (Bottom‑Up + Top‑Down + Lateral)

Bu notebook’ta **FPN (Feature Pyramid Network)**’i “ne, neden, nasıl” ekseninde adım adım anlatıyorum:

1. **FPN nedir?**
2. **Neden bulundu? (Problem: çok ölçekli nesneler)**
3. **Ne işe yarar? (Detection/segmentation vb.)**
4. **Bottom‑Up (aşağıdan yukarı) izlenimler**
5. **Top‑Down (yukarıdan aşağı) yol + lateral bağlantılar**
6. **Piramidin çıktıları (P2…P5/P6/P7)**
7. **PyTorch ile basit bir FPN implementasyonu**
8. **Pratik ayarlar, artılar/eksiler ve türevleri (PANet, BiFPN)**

> Amaç: “FPN ne yapıyor?” sorusunu kafada **net bir akış** haline getirmek. 🧠



## 1) FPN nedir?

**FPN (Feature Pyramid Network)**, bir CNN backbone’un farklı derinliklerindeki feature map’leri birleştirerek **çok ölçekli (multi‑scale) özellik piramidi** üreten mimari tasarımdır.

- CNN’in **derin katmanları**: yüksek semantik (ne olduğu), düşük çözünürlük (nerede olduğu zayıf)
- CNN’in **sığ katmanları**: yüksek çözünürlük (nerede olduğu iyi), düşük semantik (ne olduğu zayıf)

FPN, bu ikisini **top‑down yol** + **lateral (yan) bağlantılar** ile birleştirip her ölçekte **hem semantik hem konumsal** olarak güçlü feature map’ler üretir.

Kısaca:
- **Bottom‑Up**: backbone’un normal ileri akışı (çözünürlük azalır, semantik artar)
- **Top‑Down**: üst seviyeden alt seviyeye upsample + birleştirme (semantik aşağı taşınır)
- **Lateral**: aynı ölçekteki backbone özelliklerini 1×1 ile hizalayıp toplama




## 2) Neden bulundu? (Hangi problemi çözüyor?)

Nesne tespitinde (object detection) temel dert:
- Görüntüdeki nesneler **çok farklı boyutlarda** olabilir (küçük/orta/büyük).
- Tek bir feature map ölçeğiyle tespit yaparsan:
  - Büyük nesneler iyi,
  - Küçük nesneler kaçabilir (çünkü derin katmanlarda çözünürlük düşer).

Eskiden “image pyramid” (görüntüyü farklı ölçeklerde tekrar tekrar CNN’e sokmak) kullanılırdı:
- Çok pahalı (hesap maliyeti yüksek),
- Pipeline karmaşık.

**FPN**, görüntüyü tekrar ölçeklemek yerine backbone’un zaten ürettiği hiyerarşik feature’ları kullanıp **feature pyramid** üretir:
- Daha ucuz,
- Daha güçlü,
- Daha pratik.




## 3) Ne işe yarar?

FPN en çok şu işlerde kullanılır:

- **Object Detection**: RetinaNet, Faster R‑CNN (FPN’li), Mask R‑CNN
- **Instance Segmentation**: Mask R‑CNN (mask head genelde FPN seviyelerinden beslenir)
- **Keypoint / Pose**: farklı ölçeklerde daha stabil özellik çıkarımı
- Genel olarak: “çok ölçekli hedef” varsa FPN iyi bir default tercihtir ✅

Kazandırdığı şey:
- Küçük nesnelerde recall artışı
- Çok ölçekte daha dengeli temsil (semantik + detay)




## 4) Bottom‑Up (Aşağıdan Yukarı) izlenim — Backbone akışı

Backbone’u örnekleyelim (ResNet gibi düşün):

- **C2**: erken katman (yüksek çözünürlük)
- **C3**
- **C4**
- **C5**: en derin (düşük çözünürlük, yüksek semantik)

Genelde çözünürlük her seviyede 2× azalır:
- C2: 1/4
- C3: 1/8
- C4: 1/16
- C5: 1/32 (input’a göre)

**Bottom‑Up akış** şu gerçeği üretir:
- Derine indikçe “ne olduğu” güçlenir (semantik)
- Ama “nerede olduğu” zayıflar (spatial detay kaybı)

Bu yüzden sadece C5 ile detection yapmak küçük nesneleri öldürür.




## 5) Top‑Down yol + Lateral bağlantılar — FPN’in kalbi ❤️

FPN’in “büyüsü”:

### 5.1) Top‑Down (Yukarıdan Aşağı)
En üstten başlarız:
- **P5 = Conv1×1(C5)**  → kanal hizalama (ör: 256 kanal)

Sonra aşağı ineriz:
- **P4 = Up(P5) + Conv1×1(C4)**
- **P3 = Up(P4) + Conv1×1(C3)**
- **P2 = Up(P3) + Conv1×1(C2)**

Burada **Up(·)** genelde 2× nearest/bilinear upsample’dir.

### 5.2) Lateral (Yan) bağlantı
Conv1×1(Ck) şu işi yapar:
- Backbone’un farklı katmanlarında kanal sayıları farklıdır (512, 1024, 2048...)
- 1×1 conv ile hepsini aynı kanala indirip (ör 256) **toplanabilir** hale getirirsin.

**Toplama (sum)** kullanmak standarttır:
- Concatenate yaparsan kanal büyür, maliyet artar.
- Sum daha “temiz” ve baseline olarak güçlü.

### 5.3) “Smoothing” (3×3)
Toplama sonrası genelde her Pk’ya 3×3 conv konur:
- Upsample’ın getirdiği aliasing / checkerboard etkilerini azaltır,
- Daha stabil feature map verir.

Özet formül:
- **Pk = Conv3×3( Up(Pk+1) + Conv1×1(Ck) )**




## 6) FPN çıktıları: P2, P3, P4, P5 (+ P6/P7)

En sık kullanılan set:
- **P2**: en yüksek çözünürlük → küçük nesneler
- **P3**: küçük‑orta
- **P4**: orta
- **P5**: büyük nesneler

Bazı detector’lar daha da aşağı ölçek ister:
- **P6** genelde P5’ten stride=2 conv ile üretilir
- **P7** (RetinaNet) P6’dan tekrar stride=2

Bu ekstra seviyeler daha büyük receptive field hedefler.




## 7) Akışın görsel (ASCII) şeması

```bash
Bottom-Up (Backbone):
  C2 -----> C3 -----> C4 -----> C5
  (1/4)    (1/8)     (1/16)    (1/32)

Top-Down + Lateral:
              [1x1] C5 ----> P5
                         |
                        Up
                         |
              [1x1] C4 --+--> P4 --(3x3 smooth)
                         |
                        Up
                         |
              [1x1] C3 --+--> P3 --(3x3 smooth)
                         |
                        Up
                         |
              [1x1] C2 --+--> P2 --(3x3 smooth)
```

- Oklar backbone’un normal akışını (C’ler)
- Up + toplama + 3×3 ile P’ler üretilir.




## 8) PyTorch ile “minimal” FPN implementasyonu

Aşağıda basit bir FPN modülü yazıyoruz.
- Girdi: backbone’dan gelen feature’lar `[C2, C3, C4, C5]`
- Çıktı: `[P2, P3, P4, P5]`

Not:
- Burada backbone’u **hazır çıktı veriyor gibi** düşün.
- Gerçekte backbone’dan bu feature’ları almanın yolu (torchvision feature extractor vb.) var; onu da örnekleyeceğim.


In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class FPN(nn.Module):
    '''
    Minimal FPN:
      - lateral 1x1 conv: Ck -> out_channels
      - top-down upsample + sum
      - smoothing 3x3 conv
    '''
    def __init__(self, in_channels_list, out_channels=256):
        super().__init__()
        assert len(in_channels_list) == 4, "Bu minimal örnek C2..C5 (4 seviye) bekliyor."

        self.lateral_convs = nn.ModuleList([
            nn.Conv2d(cin, out_channels, kernel_size=1)
            for cin in in_channels_list
        ])
        self.smooth_convs = nn.ModuleList([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
            for _ in in_channels_list
        ])

    def forward(self, feats):
        # feats: [C2, C3, C4, C5]
        C2, C3, C4, C5 = feats

        # 1) Lateral projections
        lat2 = self.lateral_convs[0](C2)
        lat3 = self.lateral_convs[1](C3)
        lat4 = self.lateral_convs[2](C4)
        lat5 = self.lateral_convs[3](C5)

        # 2) Top-down pathway
        P5 = lat5
        P4 = lat4 + F.interpolate(P5, size=lat4.shape[-2:], mode="nearest")
        P3 = lat3 + F.interpolate(P4, size=lat3.shape[-2:], mode="nearest")
        P2 = lat2 + F.interpolate(P3, size=lat2.shape[-2:], mode="nearest")

        # 3) Smooth
        P2 = self.smooth_convs[0](P2)
        P3 = self.smooth_convs[1](P3)
        P4 = self.smooth_convs[2](P4)
        P5 = self.smooth_convs[3](P5)

        return [P2, P3, P4, P5]

# Hızlı shape testi
B = 2
C2 = torch.randn(B, 256, 128, 128)   # 1/4
C3 = torch.randn(B, 512, 64, 64)     # 1/8
C4 = torch.randn(B, 1024, 32, 32)    # 1/16
C5 = torch.randn(B, 2048, 16, 16)    # 1/32

fpn = FPN([256, 512, 1024, 2048], out_channels=256)
P2, P3, P4, P5 = fpn([C2, C3, C4, C5])
[P.shape for P in (P2, P3, P4, P5)]


[torch.Size([2, 256, 128, 128]),
 torch.Size([2, 256, 64, 64]),
 torch.Size([2, 256, 32, 32]),
 torch.Size([2, 256, 16, 16])]


Beklenen:
- P2: (B, 256, 128, 128)
- P3: (B, 256, 64, 64)
- P4: (B, 256, 32, 32)
- P5: (B, 256, 16, 16)

Kanallar eşitlendi, çözünürlük backbone seviyelerini takip etti ✅




## 9) Backbone’dan C2..C5 feature’larını nasıl alırsın? (Torchvision yaklaşımı)

Gerçek projede backbone (ResNet, CSPDarknet, EfficientNet vs.) içinden ara feature’ları almak gerekir.

Torchvision’da iki yaygın yol var:
- `torchvision.models.feature_extraction.create_feature_extractor`
- ya da modelin layer’larını manual hook’lamak

Aşağıdaki kod **örnek** amaçlıdır. Ortamda torchvision yoksa çalışmayabilir; problem değil.



In [2]:

# Bu hücre örnek amaçlıdır. torchvision ortamında çalışır.
try:
    import torchvision
    from torchvision.models.feature_extraction import create_feature_extractor

    backbone = torchvision.models.resnet50(weights=None)
    backbone.eval()

    # ResNet mapping:
    # layer1 -> C2 (stride 4), layer2 -> C3 (stride 8), layer3 -> C4 (stride 16), layer4 -> C5 (stride 32)
    return_nodes = {
        "layer1": "C2",
        "layer2": "C3",
        "layer3": "C4",
        "layer4": "C5",
    }
    extractor = create_feature_extractor(backbone, return_nodes=return_nodes)

    x = torch.randn(1, 3, 512, 512)
    with torch.no_grad():
        out = extractor(x)

    {k: v.shape for k, v in out.items()}
except Exception as e:
    print("Torchvision örneği bu ortamda çalışmadı (problem değil). Hata:", type(e).__name__, "-", e)



Bu çıktılarla FPN’i şöyle beslersin:

- C2 = out["C2"]
- C3 = out["C3"]
- C4 = out["C4"]
- C5 = out["C5"]

Sonra:
- P2..P5 = fpn([C2, C3, C4, C5])




## 10) Pratik notlar (gerçek hayatta işine yarayacak)

### 10.1) Upsample modu
- `nearest`: hızlı ve çoğu detector için yeterli
- `bilinear`: biraz daha yumuşak, ama compute artar

### 10.2) Sum vs Concat
- **Sum**: FPN’in klasik hali
- **Concat**: PANet/BiFPN gibi varyantlarda görülür (ama genelde ekstra conv gerekir)

### 10.3) Hangi seviyede hangi anchor?
Anchor-based detector’larda her P seviyesine farklı anchor scale verilir:
- P2 küçük anchor
- P5 büyük anchor

Anchor-free modellerde de (FCOS vb.) seviye ataması stride’a göre yapılır.

### 10.4) Nerede kullanılır?
- BackBone ile Head arasına.
- Detection head her Pk üzerinde aynı küçük conv stack ile çalışır.




## 11) FPN türevleri (kısa ama net)

### PANet (Path Aggregation Network)
FPN top‑down semantiği aşağı indirir.
PANet bunun üstüne bir de **bottom‑up path augmentation** ekler:
- Alt seviyeden üst seviyeye tekrar bilgi taşır (özellikle localization iyileşmesi hedeflenir).

### BiFPN (EfficientDet)
BiFPN fikri:
- Çoklu girişleri “ağırlıklı birleştir” (learnable weights)
- Gereksiz node’ları at, daha verimli grafik kur
- Tekrarlanan BiFPN blokları ile daha güçlü multi‑scale fusion




## 12) Tek paragraf özet

FPN, backbone’un bottom‑up ürettiği çok seviyeli feature’ları alır (C2..C5), üstten alta top‑down upsample yaparak semantiği aşağı indirir ve her seviyede lateral 1×1 conv ile backbone detayını ekleyip (sum) 3×3 ile temizler; sonuçta P2..P5 gibi çok ölçekli, hem semantik hem konumsal olarak güçlü feature map’ler elde eder ve bu sayede özellikle küçük nesnelerde detection performansını artırır. ✅

